In [1]:
import os
from pprint import pprint
from pdfrw import PdfReader
from pdfminer.layout import LTTextBoxHorizontal, LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfinterp import PDFTextExtractionNotAllowed
from pdfminer.converter import PDFPageAggregator, TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from io import StringIO
import unicodedata

In [2]:
def all_pdfs(src_path='examples'):
    print('=' * 80)
    print(src_path)
    pdfs = [s for s in os.listdir(src_path) if s .endswith('.pdf')]
    print(pdfs)
    return [os.path.join(src_path, e) for e in pdfs]

In [3]:
def empty_info():
    info = dict(
        title = '',
        keywords = [],
        abstract = '',
    )
    return info

In [22]:
pdf_path = 'examples\ieee_2.pdf'

pdf_path = 'C:\\Users\\liste\\OneDrive\\Documents\\schorlar\\! VisualFeatureExtractionfromVoxel-Weighted-Averaging-fMRI.pdf'

def parse_paper_ieee(pdf_path):

    info = empty_info()

    with open(pdf_path, 'rb') as fp:
        # Prepare doc
        praser = PDFParser(fp)
        doc = PDFDocument()
        praser.set_document(doc)
        doc.set_parser(praser)
        doc.initialize()
        # Rude assertion
        assert(doc.is_extractable)

        # Prepare components
        rsrcmgr = PDFResourceManager()
        laparams = LAParams()
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        # Read infos
        # Only search on first page
        page_limit = 1
        is_title = False
        for j, page in enumerate(doc.get_pages()):
            # Break when page_limit is reached
            if not(j < page_limit):
                break
            # Get layout
            interpreter.process_page(page)
            layout = device.get_result()
            # For each box
            for box in layout:
                # If not LTTextBoxHorizontal, it should be passed
                if not isinstance(box, LTTextBoxHorizontal):
                    continue
                # Get content
                content = box.get_text()
                # print('-' * 80)
                # print(content)
                if is_title:
                    info['title'] = content.replace('\n', ' ').strip()
                    is_title = False
                if content.startswith('IEEE'):
                    is_title = True
                    
                # Remove short dash '—'
                content = content.replace('—', '')
                
                # Record abstract
                if content.startswith('Abstract'):
                    content = content.replace('Abstract', '')
                    info['abstract'] = content.replace('\n', ' ').replace('- ', '').strip()
                    
                # Record keywords
                content = content.replace('Index Terms', 'IndexTerms')
                if content.startswith('IndexTerms'):
                    keywords = content.replace('IndexTerms', '').replace('\n', '').split(',')
                    if keywords[-1].endswith('.'):
                        keywords[-1] = keywords[-1][:-1]
                    info['keywords'] = [e.strip() for e in keywords]
    return info

parse_paper_ieee(pdf_path)

{'title': 'Visual Feature Extraction From Voxel-Weighted Averaging of Stimulus Images in 2 fMRI Studies',
 'keywords': ['Bayesian estimation',
  'component analysis',
  'fMRI',
  'generalized linear models',
  'imaging',
  'voxel'],
 'abstract': 'Multiple studies have provided evidence for distributed object representation in the brain, with several recent experiments leveraging basis function estimates for partial image reconstruction from fMRI data. Using a novel combination of statistical decomposition, generalized linear models, and stimulus averaging on previously examined image sets and Bayesian regression of recorded fMRI activity during presentation of these data sets, we identify a subset of relevant voxels that appear to code for covarying object features. Using a technique we term “voxel-weighted averaging,” we isolate image ﬁlters that these voxels appear to implement. The results, though very cursory, appear to have signiﬁcant implications for hierarchical and deep-learnin

In [29]:
pdf_path = 'examples\elsevier_2.pdf'
pdf_path = 'C:\\Users\\liste\\OneDrive\\Documents\\schorlar\\!Improved estimation of subject-level functional connectivity using full and partial correlation with empirical Bayes shrinkage.pdf'
pdf_path = 'C:\\Users\\liste\\OneDrive\\Documents\\schorlar\\1-s2.0-S1053811916301914-main.pdf'
pdf_path = 'C:\\Users\\liste\\OneDrive\\Documents\\schorlar\\10.1093@cercor@bhy123.pdf'
pdf_path = 'C:\\Users\\liste\\OneDrive\\Documents\\schorlar\\1-s2.0-S0896627306005861-main.pdf'
pdf_path = 'C:\\Users\\liste\\OneDrive\\Documents\\schorlar\\1-s2.0-S0028393216300173-main.pdf'

def parse_paper_elsevier(pdf_path):
    info = empty_info()
    info['title'] = PdfReader(pdf_path).Info['/Title'][1:-1]

    with open(pdf_path, 'rb') as fp:
        # Prepare doc
        praser = PDFParser(fp)
        doc = PDFDocument()
        praser.set_document(doc)
        doc.set_parser(praser)
        doc.initialize()
        # Rude assertion
        assert(doc.is_extractable)

        # Prepare components
        rsrcmgr = PDFResourceManager()
        laparams = LAParams()
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        # Read infos
        # Only search on first page
        page_limit = 2
        is_abstract = False
        for j, page in enumerate(doc.get_pages()):
            # Break when page_limit is reached
            if not(j < page_limit):
                break
            # Get layout
            interpreter.process_page(page)
            layout = device.get_result()
            # For each box
            for box in layout:
                # If not LTTextBoxHorizontal, it should be passed
                if not isinstance(box, LTTextBoxHorizontal):
                    continue
                # Get content
                content = box.get_text()
                # print('-' * 80)
                # print(content)
                # Record abstract
                if all([is_abstract,
                        not content.startswith('Article history'),
                        not content.startswith('Keywords'),
                        not content.startswith('Introduction'),
                        not content.startswith('1.')
                       ]):
                    info['abstract'] = ''.join(content).replace('\n', ' ').strip()
                    is_abstract = False
                if any([content.lower().startswith('a b s t r a c t'),
                        content.lower().startswith('abstract'),
                        content.lower().startswith('summary')]):
                    is_abstract = True
                # Record keywords
                if content.startswith('Keywords'):
                    info['keywords'] = [e for e in content.split('\n') if e][1:]
                if content.startswith('Key words'):
                    info['keywords'] = [e.strip() for e in content.replace(',', '\n').split('\n') if e][1:]

    return info

parse_paper_elsevier(pdf_path)

{'title': 'A meta-analysis of fMRI decoding_ Quantifying influences on human visual population codes',
 'keywords': ['Decoding',
  'MVPA',
  'Patterns',
  'Meta-analysis',
  'Vision',
  'Objects'],
 'abstract': 'Information in the human visual system is encoded in the activity of distributed populations of neurons, which in turn is reﬂected in functional magnetic resonance imaging (fMRI) data. Over the last ﬁfteen years, activity patterns underlying a variety of perceptual features and objects have been decoded from the brains of participants in fMRI scans. Through a novel multi-study meta-analysis, we have analyzed and modeled relations between decoding strength in the visual ventral stream, and stimulus and methodological variables that differ across studies. We report ﬁndings that suggest: (i) several organi- zational principles of the ventral stream, including a gradient of pattern granulation and an increasing abstraction of neural representations as one proceeds anteriorly; (ii) 

In [24]:
def pdf_info(pdf_path):
    info = PdfReader(pdf_path).Info
    if info['/Creator'] == '(Elsevier)':
        return _elsevier(pdf_path)
    return _ieee(pdf_path)
    return None

def _elsevier(pdf_path):
    print('[-- It is an elsevier paper. --]')
    info = parse_paper_elsevier(pdf_path)
    return info

def _ieee(pdf_path):
    print('[-- It is an IEEE paper. --]')
    info = parse_paper_ieee(pdf_path)
    return info

In [25]:
for pdf_path in all_pdfs('C:\\Users\\liste\\OneDrive\\Documents\\schorlar'):
    print('-'* 80)
    print(pdf_path)
    info = pdf_info(pdf_path)
    if info:
        pprint(info)

C:\Users\liste\OneDrive\Documents\schorlar
['! VisualFeatureExtractionfromVoxel-Weighted-Averaging-fMRI.pdf', '!10.1093@cercor@bhy123.pdf', '!Improved estimation of subject-level functional connectivity using full and partial correlation with empirical Bayes shrinkage.pdf', '!Integrating theoretical models with functional neuroimaging.pdf', '0102181v1.pdf', '07073970.pdf', '07264993.pdf', '07468569.pdf', '08025614.pdf', '08051252.pdf', '08269806.pdf', '08370896.pdf', '1-s2.0-S0010945217303258-main.pdf', '1-s2.0-S0028393217300593-main.pdf', '1-s2.0-S0042698909004751-main.pdf', '1-s2.0-S0893608016301800-main.pdf', '1-s2.0-S0896627306005861-main.pdf', '1-s2.0-S089662731830477X-main.pdf', '1-s2.0-S0925231215017634-main.pdf', '1-s2.0-S1053810015000355-main.pdf', '1-s2.0-S1053811907001073-main.pdf', '1-s2.0-S1053811911013012-main.pdf', '1-s2.0-S1053811912004429-main.pdf', '1-s2.0-S1053811913002851-main.pdf', '1-s2.0-S1053811914001220-main.pdf', '1-s2.0-S1053811914010325-main.pdf', '1-s2.0-S1


[-- It is an elsevier paper. --]
{'abstract': 'Reliability of subject-level resting-state functional '
             'connectivity (FC) is determined in part by the statistical '
             'techniques employed in its estimation. Methods that pool '
             'information across subjects to inform estimation of '
             'subject-level effects (e.g., Bayesian approaches) have been '
             'shown to enhance reliability of subject-level FC. However, fully '
             'Bayesian approaches are computationally demanding, while '
             'empirical Bayesian approaches typically rely on using repeated '
             'measures to estimate the variance components in the model. Here, '
             'we avoid the need for repeated measures by proposing a novel '
             'measurement error model for FC describing the different sources '
             'of variance and error, which we use to perform empirical Bayes '
             'shrinkage of subject-level FC towards th

{'abstract': 'Linear Gaussian state-space models are ubiquitous in signal '
             'processing, and an important procedure is that of estimating '
             'system parameters from observed data. Rather than making a '
             'single point estimate, it is often desirable to conduct Bayesian '
             'learning, in which the entire posterior distribution of the '
             'unknown parameters is sought. This can be achieved using Markov '
             'chain Monte Carlo. On some occasions it is possible to deduce '
             'the form of the unknown system matrices in terms of a small '
             'number of scalar parameters, by considering the underlying '
             'physical processes involved. Here we study the case where this '
             'is not possible, and the entire matrices must be treated as '
             'unknowns. An efﬁcient Gibbs sampling algorithm exists for the '
             'basic formulation of linear model. We extend this to the mo

{'abstract': 'Principal component analysis (PCA) is an exploratory tool widely '
             'used in data analysis to uncover the dominant patterns of '
             'variability within a population. Despite its ability to '
             'represent a data set in a low-dimensional space, PCA ’s '
             'interpretability remai ns limited. Indeed, the components '
             'produced by PCA are often noisy or exhibit no visually '
             'meaningful patterns. Furthermore, the fact that the components '
             'are usually non-sparse may also impede interpretation, unless '
             'arbitrary thresholding is applied. However, in neuroimaging, it '
             'is essential to uncover clinically interpretable phenotypic '
             'markers that would account for the main variability in the brain '
             'images of a population. Recently, some alternatives to the '
             'standard PCA approach, such as sparse PCA (SPCA), have been '
           

{'abstract': '',
 'keywords': ['Functional magnetic resonance imaging data analysis; Dynamic '
              'analysis'],
 'title': 'Dynamic discrimination analysis: A spatialâ\xa0\x8dtemporal SVM'}
--------------------------------------------------------------------------------
C:\Users\liste\OneDrive\Documents\schorlar\1-s2.0-S1053811911013012-main.pdf
[-- It is an elsevier paper. --]


...

{'abstract': '1. The basic idea: all is contextual',
 'keywords': [],
 'title': 'Top-down effects in the brain'}
--------------------------------------------------------------------------------
C:\Users\liste\OneDrive\Documents\schorlar\Toward Universal Lingustic Encoding.pdf
[-- It is an IEEE paper. --]
{'abstract': '', 'keywords': [], 'title': ''}
--------------------------------------------------------------------------------
C:\Users\liste\OneDrive\Documents\schorlar\TowardIntegrationDNN-Neuroscience.pdf
[-- It is an IEEE paper. --]
{'abstract': '', 'keywords': [], 'title': ''}
--------------------------------------------------------------------------------
C:\Users\liste\OneDrive\Documents\schorlar\Transfer learning of deep neural network representations for fMRI decoding.pdf
[-- It is an IEEE paper. --]
{'abstract': '', 'keywords': [], 'title': ''}
--------------------------------------------------------------------------------
C:\Users\liste\OneDrive\Documents\schorlar\Visual pa

{'abstract': 'Vision research has been shaped by the seminal insight that we '
             'can understand the higher-tier visual cortex from the '
             'perspective of multiple functional pathways with different '
             'goals. In this paper, we try to give a computational account of '
             'the functional organization of this system by reasoning from the '
             'perspective of multi-task deep neural networks. Machine learning '
             'has shown that tasks become easier to solve when they are '
             'decomposed into subtasks with their own cost function. We '
             'hypothesize that the visual system optimizes multiple cost '
             'functions of unrelated tasks and this causes the emergence of a '
             'ventral pathway dedicated to vision for perception, and a dorsal '
             'pathway dedicated to vision for action. To evaluate the '
             'functional organization in multi-task deep neural networks, we '

{'abstract': '',
 'keywords': [],
 'title': 'What is changing when: Decoding visual information in movies from '
          'human intracranial recordings'}
--------------------------------------------------------------------------------
C:\Users\liste\OneDrive\Documents\schorlar\What When Where Visual Word Recog-TICS2014.pdf
[-- It is an IEEE paper. --]


{'abstract': '', 'keywords': [], 'title': ''}
--------------------------------------------------------------------------------
C:\Users\liste\OneDrive\Documents\schorlar\zeilerECCV2014.pdf
[-- It is an IEEE paper. --]
{'abstract': '. Large Convolutional Network models have recently demonstrated '
             'impressive classiﬁcation performance on the ImageNet benchmark '
             'Krizhevsky et al. [18]. However there is no clear understanding '
             'of why they perform so well, or how they might be improved. In '
             'this paper we explore both issues. We introduce a novel '
             'visualization technique that gives insight into the function of '
             'intermediate feature layers and the operation of the classiﬁer. '
             'Used in a diagnostic role, these visualizations allow us to ﬁnd '
             'model architectures that outperform Krizhevsky et al. on the '
             'ImageNet classiﬁcation benchmark. We also perform an ablation

{'abstract': 'A principal goal in sensory neuroscience is to understand how '
             'properties of our environment are reflected in neural activity '
             'patterns. Recent advances in computational modeling provide '
             'increasingly accurate predictions of how neural populations '
             'across the brain respond to complex naturalistic stimuli. The '
             'em- ployed computational models, referred to as encoding models, '
             'explicitly transform complex stimuli into observed neural '
             'responses. This rapidly developing field is becoming '
             'increasingly important in sensory neuroscience as it provides '
             'detailed insights into the functional organization of neural '
             'representations. The present work starts by discussing the '
             'theoretical underpinnings of encoding models. Next, various ap- '
             'plications of encoding models are reviewed. Finally, potential '
